In [1]:
import os
from fastai.text import * 

In [2]:
torch.cuda.device_count()

1

In [3]:
LANG = 'en'

In [4]:
proj_path = Path(os.getcwd()).parents[1]

In [5]:
data_path = proj_path/f'clean_data_csv_{LANG}'

In [6]:
df = pd.read_csv(data_path/'all_bot_human_data.csv')

In [7]:
df.head()

,bot_human,text
0,human,RT @GeriMilk: Excited to be back New York to s...
1,bot,Spring is when you feel like whistling even wi...
2,bot,Anyone can run 20 miles. It's the next six tha...
3,bot,Great overview on using Redstone in Minecraft ...
4,human,@comedy_girl Whaaat? Big news!


In [8]:
data_lm = TextLMDataBunch.from_csv(data_path, 'all_bot_human_data.csv')

In [9]:
data_clas = TextClasDataBunch.from_csv(data_path, 'all_bot_human_data.csv', 
                                       vocab=data_lm.train_ds.vocab, bs=32)

In [10]:
data_lm.save(f'data_lm_export_{LANG}.pkl')
data_clas.save(f'data_clas_export_{LANG}.pkl')

In [11]:
data_lm = load_data(data_path, fname=f'data_lm_export_{LANG}.pkl')
data_clas = load_data(data_path, fname=f'data_clas_export_{LANG}.pkl', bs=16)

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(3, 1e-2)

epoch,train_loss,valid_loss,accuracy,time


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

In [ ]:
learn.predict("This is a review about", n_words=10)

In [ ]:
learn.save_encoder('ft_enc')

In [ ]:
learn_class = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn_class.load_encoder('ft_enc')

In [ ]:
data_clas.show_batch()

In [ ]:
learn_class.fit_one_cycle(5, 1e-2)